In [ ]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Download a Postgres driver that will allow Spark to interact with Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-05-02 21:01:49--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.05MB/s    in 0.8s    

2020-05-02 21:01:50 (1.05 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
# Start a Spark session with the additional option that adds the driver to Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("M16_Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

**EXTRACT** : Connect to data storage, then extract that data into a DataFrame.

In [ ]:
# Read in video games dataset from S3 Buckets
from pyspark import SparkFiles

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
video_games_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
video_games_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...|2015-08-31 00:00:00|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...| 

In [ ]:
# Count the number of records in the Video Games DataFrame
video_games_df.count()

1785997

**TRANSFORM**: Now that the raw data stored in S3 is available in a PySpark DataFrame, perform transformations.

In [ ]:
# Drop null values
dropna_df = video_games_df.dropna()
dropna_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...|2015-08-31 00:00:00|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...| 

In [ ]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

In [ ]:
# Drop duplicates
clean_df = dropna_df.drop_duplicates()
clean_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   20009611|R100E0YCS06YNR|B00EFFW0HC|     277685961|       Battlefield 4|     Video Games|          5|            0|          0|   N|                Y|          Five Stars|     great thank you|2015-07-04 00:00:00|
|         US|   21036809|R100HR44BSB9I4|B007FTE2VW|     265303108|SimCity - Limited...| 

In [ ]:
# Load in a sql function to us to_date
from pyspark.sql.functions import to_date

In [ ]:
# Create the review_id DataFrame
review_id_df = clean_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100E0YCS06YNR|   20009611|B00EFFW0HC|     277685961| 2015-07-04|
|R100HR44BSB9I4|   21036809|B007FTE2VW|     265303108| 2013-03-07|
|R100UABZOUF1JP|    4444018|B005HN5LKY|     148114874| 2015-08-17|
|R101MOW8A5WC4J|   51581032|B009LEIUY4|     563376217| 2014-08-16|
|R101X7KSBGIWF1|   41543132|B00004UC1W|     940154393| 2014-12-28|
|R101ZTSTDC8KQ4|    2074450|B006WN5UM2|     304022369| 2014-03-10|
|R103Q4U55JZAD1|   11407363|B0025KZV7A|     130147097| 2011-09-06|
|R103SW6BLR1HH6|   10547153|B004ACGO0Y|     540734971| 2011-09-25|
|R1048AZPZ1XQVY|   15338940|B005XTRC7M|     325268333| 2015-01-22|
|R104C56FZRE21Q|   49041788|B001US6O2Y|     855199089| 2015-05-20|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [ ]:
# Create the products DataFrame and drop the duplicates
products_df = clean_df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B000099T1Y|Jak II - PlayStat...|
|B002L8W5V6|Dotop Nintendo Ga...|
|B00KVOVBGM|PlayStation 4 Con...|
|B00002EPZ2|Planescape: Torme...|
|B007ZHCZTA|Bioworld Nintendo...|
|B00139PR5U|The Chronicles of...|
|B000QAVV6Q|The Sims 2: Casta...|
|B0001UEHP8|Fable: The Lost C...|
|B00CJ7IUI6|The Elder Scrolls...|
|B00DHF39KS|Wolfenstein: The ...|
+----------+--------------------+
only showing top 10 rows



In [ ]:
# Create the customers DataFrame
customers_df = clean_df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   16505131|             1|
|   17235913|             1|
|   17782326|             2|
|   40985731|             7|
|   31554643|             1|
|   47501193|             7|
|    6367756|            65|
|   45401845|             1|
|   42066493|             8|
|    5483518|             1|
+-----------+--------------+
only showing top 10 rows



In [ ]:
# Create the vine DataFrame
vine_df = clean_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100E0YCS06YNR|          5|            0|          0|   N|
|R100HR44BSB9I4|          1|            1|          1|   N|
|R100UABZOUF1JP|          1|            0|          0|   N|
|R101MOW8A5WC4J|          5|            0|          0|   N|
|R101X7KSBGIWF1|          1|            0|          0|   N|
|R101ZTSTDC8KQ4|          5|            0|          0|   N|
|R103Q4U55JZAD1|          5|            0|          0|   N|
|R103SW6BLR1HH6|          5|            0|          0|   N|
|R1048AZPZ1XQVY|          4|            0|          0|   N|
|R104C56FZRE21Q|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



**LOAD**: Get the transformed raw data into database. PySpark can easily connect to a database to load the DataFrames into the table.

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://challenge-db.cju19akozuui.us-west-1.rds.amazonaws.com:5432/challenge-db"
config = {"user":"###", 
          "password": "###", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write dataFrame to review_id table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write dataframe to products table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write dataframe to customers table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write dataframe to vine table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)